In [1]:
import pandas as pd
import os

In [2]:
filefolder = r'C:\Users\johan\OneDrive\Marcobre\1.0 PROJECTS\01. Crear budget de mantenimiento\hr2csv'
filename = r'5._Carga de hoja de ruta - GRUPO 5.xlsx'

In [3]:
filerute = os.path.join(filefolder,filename)

In [4]:
db = pd.read_excel(filerute,sheet_name=0, skiprows = 6, header = None)

In [5]:
# Hojas de ruta
THR = db.loc[db.iloc[:,0] == 1,:]

In [6]:
THR.head(12)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,1,MBOM020,M1,INSP_MEC_BOMBA_SUMMIT 2196MTO-7,TEC-MEC,JP11,4.0,P01,4.0,1.0,HOR002,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,1,MBOM020,M2,PREV_MEC_BOMBA_SUMMIT 2196MTO-7,TEC-MEC,JP11,4.0,P01,4.0,1.0,HOR002,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,1,MBOM020,E1,INSP_ELE_BOMBA_SUMMIT 2196MTO-7,TEC-ELE,JP11,4.0,P01,4.0,1.0,CAL001,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,1,MBOM020,E2,PREV_ELE_BOMBA_SUMMIT 2196MTO-7,TEC-ELE,JP11,4.0,P01,4.0,1.0,HOR002,NaN,NaN,NaN,NaN,NaN,NaN,NaN
143,1,MBOM020,I1,INSP_INS_BOMBA_SUMMIT 2196MTO-7,TEC-INS,JP11,4.0,P01,4.0,1.0,CAL001,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151,1,MBOM020,I2,PREV_INS_BOMBA_SUMMIT 2196MTO-7,TEC-INS,JP11,4.0,P01,4.0,1.0,HOR002,NaN,NaN,NaN,NaN,NaN,NaN,NaN
160,1,MBOM020,P1,PRED_CON_BOMBA_SUMMIT 2196MTO-7,TEC-PRED,JP11,4.0,P01,4.0,1.0,HOR002,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193,1,MBOM021,M1,INSP_MEC_BOMBA_GOULDS 23SHK6,TEC-MEC,JP11,4.0,P01,4.0,1.0,HOR002,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209,1,MBOM021,M2,PREV_MEC_BOMBA_GOULDS 23SHK6,TEC-MEC,JP11,4.0,P01,4.0,1.0,HOR002,NaN,NaN,NaN,NaN,NaN,NaN,NaN
252,1,MBOM021,E1,INSP_ELE_BOMBA_GOULDS 23SHK6,TEC-ELE,JP11,4.0,P01,4.0,1.0,CAL001,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Posiciones de las Hojas de Ruta
PHR = db.loc[db.iloc[:,0] == 2,:]
PHR.head(12)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
1,2,MBOM020,10,TEC-MEC,PM01,Inspeccion mantenimiento de acople 2000H,1.0,H,1.0,1.0,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2,MBOM020,20,TEC-MEC,PM01,Inspeccion general bomba 330H,0.8,H,2.0,0.4,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,2,MBOM020,30,TEC-MEC,PM01,Inspeccion general bomba 660H,1.2,H,2.0,0.6,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,2,MBOM020,10,TEC-MEC,PM01,Cambio de impulsor 4000H,18.0,H,3.0,6.0,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,2,MBOM020,20,TEC-MEC,PM01,Cambio de sello mecanico 8000H,6.0,H,2.0,3.0,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,2,MBOM020,30,TEC-MEC,PM01,Cambio de eje 8000H,10.0,H,2.0,5.0,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,2,MBOM020,40,TEC-MEC,PM01,Cambio de elementos de union 8000H,2.0,H,2.0,1.0,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56,2,MBOM020,50,TEC-MEC,PM01,Mtto general carcasa 8000H,36.0,H,2.0,18.0,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,2,MBOM020,10,TEC-ELE,PM01,Inspeccion general del motor 2S,1.0,H,2.0,0.5,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,2,MBOM020,10,TEC-ELE,PM01,Mantenimiento menor de motor 2000H,4.0,H,2.0,2.0,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Actividad detallada de laPosiciones de las Hojas de Ruta
AHR = db.loc[db.iloc[:,0] == 3,:]
AHR.head(12)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
2,3,MBOM020,Inspeccion mantenimiento de acople 2000H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,MBOM020,Verificar ruido inusual y/o solturas en los el...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,MBOM020,"acople, ajustar elementos de union y reemplaza...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,3,MBOM020,Inspeccion general bomba 330H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,3,MBOM020,* Inspeccion de caja de rodamientos 330H: Veri...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,3,MBOM020,"ruidos ni vibraciones inusuales, verificar la ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,3,MBOM020,"80°C, verificar fugas de grasa por los sellos ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,3,MBOM020,* Inspeccion de sello mecanico 330H: Verificar...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,3,MBOM020,de ser necesario realizar el cambio del kit de...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,3,MBOM020,total del sello.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
